# Introduction to py-sdk

The python sdk allows you to create queries and transaction to a Nibiru chain.

It allows to interact with all of the modules and can be leveraged to automate trading strategies or monitor them.

This notebook will guide you on opening and closing positions.

In [17]:
import json

from nibiru import Network, ChainClient, Msg, Coin

Let's set up the chain to which we will be connecting to. If you have a preferred endpoint you can update these values to connect to it. Make sure that the node have the port opens for 1317, 26657 and 9090.

In [4]:
pair = "ubtc:unusd"

Next, let's make sure that the transaction we send are synchronous. This way if no error is raised when running the execute_msgs method, we are sure that the transaction was successful.

In [8]:
mnemonic = "exclude rural knife asset egg seminar carry control thought stock topic lens embark about huge obvious reason language ripple clean cousin library punch renew"
client = ChainClient(network=Network.testnet(2))
client.authenticate(mnemonic=mnemonic)

# Check balance
print(client.query.get_bank_balances(client.address))

{'balances': [{'denom': 'unibi', 'amount': 99988614936458}, {'denom': 'unusd', 'amount': 1000000165415947734}, {'denom': 'uusdt', 'amount': 10000000200000000}], 'pagination': {'next_key': b'', 'total': 3}}


Now with those fresh token, we can open a long position

In [12]:
output = client.tx.execute_msgs(
    Msg.perp.open_position(
        sender=client.address,
        pair=pair,
        is_long=True,
        quote_asset_amount=10,
        leverage=1,
        base_asset_amount_limit=0,
    )
)
print(output)

ExecuteTxResp(code=0, tx_hash='946A4252423A0AC183C1BAA2B76CCB25512DC9A18861D8782EFE7F14512CBDF6', log='[]')


If your TX has succeeded, you should get output something like:
ExecuteTxResp(code=0, tx_hash='946A4252423A0AC183C1BAA2B76CCB25512DC9A18861D8782EFE7F14512CBDF6', log='[]')
Query your TX hash to get TX details:

In [13]:
output = client.query.tx_by_hash("946A4252423A0AC183C1BAA2B76CCB25512DC9A18861D8782EFE7F14512CBDF6")
print(output)

{'tx': {'body': {'messages': [{'type_url': '/nibiru.perp.v2.MsgMarketOrder', 'value': [10, 43, 110, 105, 98, 105, 49, 106, 108, 101, 56, 107, 104, 106, 51, 97, 101, 110, 110, 113, 50, 52, 122, 120, 54, 103, 57, 51, 97, 97, 109, 57, 114, 116, 48, 102, 113, 104, 103, 121, 112, 52, 104, 53, 50, 18, 10, 117, 98, 116, 99, 58, 117, 110, 117, 115, 100, 24, 1, 34, 2, 49, 48, 42, 19, 49, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 50, 1, 48]}], 'memo': '', 'timeout_height': 902379, 'extension_options': [], 'non_critical_extension_options': []}, 'auth_info': {'signer_infos': [{'public_key': {'type_url': '/cosmos.crypto.secp256k1.PubKey', 'value': [10, 33, 3, 49, 132, 132, 31, 74, 77, 38, 216, 26, 253, 113, 231, 178, 209, 102, 161, 168, 243, 31, 103, 203, 239, 112, 166, 177, 205, 144, 9, 244, 227, 62, 253]}, 'mode_info': {'single': {'mode': 1}, 'multi': {'bitarray': {'extra_bits_stored': 0, 'elems': b''}, 'mode_infos': []}}, 'sequence': 18411}], 'fee': {'amount': [{'de

Or directly query the state of your position:

In [14]:
position = client.query.perp.position(pair, client.address)
print(json.dumps(position, indent=4))

{
    "position": {
        "trader_address": "nibi1jle8khj3aennq24zx6g93aam9rt0fqhgyp4h52",
        "pair": "ubtc:unusd",
        "size": 0.000382279288570798,
        "margin": 10.0,
        "open_notional": 10.0,
        "latest_cumulative_premium_fraction": -2025.592974347103,
        "last_updated_block_number": 902360
    },
    "position_notional": 10.000713635244779,
    "unrealized_pnl": 0.000713635244778446,
    "margin_ratio": 1.0
}


We can then add margin to our position:

In [18]:
output = client.tx.execute_msgs(
    Msg.perp.add_margin(
        sender=client.address,
        pair=pair,
        margin=Coin(1, "unusd"),
    )
)
print(output)

ExecuteTxResp(code=0, tx_hash='5C94F2809960449438416AF5D0CABC13AEE4A750BE221B5EBF4DECE9272498D9', log='[]')


Or remove margin:

In [ ]:
output = client.tx.execute_msgs(
    Msg.perp.remove_margin(
        sender=client.address,
        pair=pair,
        margin=Coin(1, "unusd")
    )
)
print(output)

We can then totally close the position:

In [19]:
output = client.tx.execute_msgs(
    Msg.perp.close_position(
        sender=client.address,
        pair=pair,
    )
)
print(output)

ExecuteTxResp(code=0, tx_hash='6B7B232205F512C98A6125DF9EF93E660474F15D7D4BCE138A9AF661FE68628B', log='[]')


## Multi-transactions

We can build multiple messages into a single transactions to be sure that they are executed consecutively.

It can be useful for example to send tokens after removing margins from a position

Lets create a transction with:
- Agent open a position
- Agent add margin
- Agent close the position

In [20]:
output = client.tx.execute_msgs(
    [
        Msg.perp.open_position(
            sender=client.address,
            pair=pair,
            is_long=True,
            quote_asset_amount=10,
            leverage=1,
            base_asset_amount_limit=0,
        ),
        Msg.perp.add_margin(
            sender=client.address,
            pair=pair,
            margin=Coin(1, "unusd"),
        ),
        Msg.perp.close_position(
            sender=client.address,
            pair=pair,
        ),
    ]
)
print(output)

ExecuteTxResp(code=0, tx_hash='E697AFD9B63B3C6C312B9707B600F437C3A4CBFCA20D5D853E067BB40931B634', log='[]')
